Once we have chosen an architecture and set our hyper-parameters, we proceed to the training loop, where our goal is to
find parameter values that minimize our loss function. After training, we will need these parameters in order to make
future predictions. Additionally, we will sometimes wish to extract the parameters either to reuse them in some other
context, to save our model to disk so that it may be executed in other software, or for examination in the hope of
gaining scientific understanding.

Most of the time, we will be able to ignore the nitty-gritty details of how parameters are declared and manipulated,
relying on deep learning frameworks to do the heavy lifting. However, when we move away from stacked architectures with
standard layers, we will sometimes need to get into the weeds of declaring and manipulating parameters.

    1. Accessing parameters for debugging, diagnostics, and visualizations.
    2. Sharing parameters across different model components.

We start by focusing on an MLP with one hidden layer.


In [2]:

import torch
from torch import nn

net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
print(net(X).shape)

torch.Size([2, 1])


## Parameter Access
hen a model is defined via the Sequential class, we can first access any layer by indexing into the model as though it
were a list. Each layer’s parameters are conveniently located in its attribute. We can inspect the parameters of the
second fully connected layer as follows.

In [9]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [8]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.3428,  0.0276,  0.4506, -0.0477],
                      [ 0.1517,  0.3550,  0.0031,  0.0775],
                      [-0.3637, -0.3808,  0.2689,  0.1316],
                      [ 0.2998,  0.4899,  0.0136, -0.2430],
                      [-0.1342, -0.0804,  0.4399,  0.2467],
                      [-0.0521,  0.4535, -0.3810, -0.1987],
                      [-0.0645, -0.1014,  0.4848,  0.3358],
                      [ 0.1339, -0.0184,  0.2427,  0.3787]])),
             ('0.bias',
              tensor([-0.2302,  0.3023,  0.2565, -0.2527, -0.3305,  0.3979,  0.2156,  0.2248])),
             ('2.weight',
              tensor([[ 0.1899, -0.0977,  0.3370,  0.3385,  0.2369, -0.0122,  0.2197, -0.3393]])),
             ('2.bias', tensor([-0.1352]))])

In [12]:
print(">>> net 2: ", net[2])
print("net 2 parameters: \n", net[2].state_dict())

>>> net 2:  Linear(in_features=8, out_features=1, bias=True)
net 2 parameters: 
 OrderedDict([('weight', tensor([[ 0.1899, -0.0977,  0.3370,  0.3385,  0.2369, -0.0122,  0.2197, -0.3393]])), ('bias', tensor([-0.1352]))])


### Targeted Parameters

Note that each parameter is represented as an instance of the parameter class. To do anything useful with the parameters, we first need to access the underlying numerical values. There are several ways to do this. Some are simpler while others are more general. The following code extracts the bias from the second neural network layer, which returns a parameter class instance, and further accesses that parameter’s value.

In [15]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.1352]))

In [16]:
print("Parameters are represented as an instance of Parameter class: \n", type(net[2].bias))
print('Its data is tensor: ',net[2].bias.data)

Parameters are represented as an instance of Parameter class: 
 <class 'torch.nn.parameter.Parameter'>
Its data is tensor:  tensor([-0.1352])


Parameters are complex objects, containing values, gradients, and additional information. That’s why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.

In [19]:
net[2].weight.grad is None

True

### All Parameters at Once
When we need to perform operations on all parameters, accessing them one-by-one can grow tedious. The situation can grow especially unwieldy when we work with more complex modules (e.g., nested modules), since we would need to recurse through the entire tree to extract each sub-module’s parameters. Below we demonstrate accessing the parameters of all layers.

In [22]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## Tied Parameters
Often, we want to share parameters across multiple layers. Let’s see how to do this elegantly. In the following we allocate a fully connected layer and then use its parameters specifically to set those of another layer. Here we need to run the forward propagation net(X) before accessing the parameters.

In [23]:
shared = nn.LazyLinear(8)
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)

A:\huan_shit\Study_Shit\Deep_Learning\Dive_into_Deep_Learning\venv\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[-0.0311],
        [-0.0267]], grad_fn=<AddmmBackward0>)

In [24]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)

In [26]:
# Check whether the parameters are the same
net[2].weight.data[0] == net[4].weight.data[0]


tensor([True, True, True, True, True, True, True, True])

In [28]:
# Make sure that they are actually the same object rather than just having the
# same value
net[2].weight.data[0, 0] = 100

In [29]:
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters of the second and third layer are tied. They are not just equal, they are represented by the same exact tensor. Thus, if we change one of the parameters, the other one changes, too. You might wonder, when parameters are tied what happens to the gradients? Since the model parameters contain gradients, the gradients of the second hidden layer and the third hidden layer are added together during backpropagation.

## Summary
We have several ways to access and tie model parameters.